# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2021'
#y0 = '2013'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400112','0400129']

#north_pumas = ['0400112','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP'

ADJHSG - adjustment factor for housing dollar amounts (6 decimal places)
1000000 = 1.000000

FHINCP - income flag - 1: yes | https://api.census.gov/data/2021/acs/acs1/pums/variables/FHINCP.json

HINCP - HHI in past 12 months (not -60000:n/a, 0:no income, -59999 loss of 59k+)
-1 to -59998 loss, 1+
https://api.census.gov/data/2021/acs/acs1/pums/variables/HINCP.json

GRNTP - gross rent (monthly) (not 0: n/a,not paying rent, owned)
https://api.census.gov/data/2021/acs/acs1/pums/variables/GRNTP.json

In [4]:
#AMI bands in 2021 by household size
AMI_30pct = {'1':16600,'2':19000,'3':21960,'4':26500,'5':31040,'6':35580,\
            '7':40120,'8':44660}
AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71100,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [5]:
#Housing costs affordable to different AMI bands based on household size
aff_1p = {'30pct':461,'50pct':767,'80pct':1217,'100pct':1535}
aff_2p = {'30pct':527,'50pct':877,'80pct':1392,'100pct':1754}
aff_3p = {'30pct':609,'50pct':985,'80pct':1565,'100pct':1973}
aff_4p = {'30pct':735,'50pct':1096,'80pct':1738,'100pct':2192}
aff_5p = {'30pct':861,'50pct':1185,'80pct':1878,'100pct':2370}
aff_6p = {'30pct':987,'50pct':1272,'80pct':2017,'100pct':2545}
aff_7p = {'30pct':1113,'50pct':1360,'80pct':2156,'100pct':2720}
aff_8p = {'30pct':1239,'50pct':1447,'80pct':2295,'100pct':2894}

unit_afford = {'1':[0,461,767,1217,1535,1000000000],\
              '2':[0,527,877,1392,1535,1000000000],\
              '3':[0,609,985,1565,1973,1000000000],\
              '4':[0,735,1096,1738,2192,1000000000],\
              '5':[0,861,1185,1878,2370,1000000000],\
              '6':[0,987,1272,2017,2545,1000000000],\
              '7':[0,1113,1360,2156,2720,1000000000],\
              '8':[0,1239,1447,2295,2894,1000000000]}

inc_lbls = ['u30_ami','30_50_ami','50_80_ami','80_100_ami','o100_ami']

In [6]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [7]:
df = get.get_puma(sample,y1,data_cols)

In [8]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [9]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)
df['HINCP'] = df.ADJINC * df.HINCP

In [10]:
dff = df[(df.GRNTP!=0)].copy()

In [11]:
dff['hou_cost'] = dff.GRNTP
dff['aff_cost'] = np.where(dff.HINCP>=1,(dff.HINCP*0.333)/12,0)

In [12]:
#who is the unit affordable to based on the rent
dff['unit_aff'] = np.where(dff.HHSz=='1',pd.cut(dff['hou_cost'],bins=unit_afford['1'],labels=inc_lbls),\
                  np.where(dff.HHSz=='2',pd.cut(dff['hou_cost'],bins=unit_afford['2'],labels=inc_lbls),\
                  np.where(dff.HHSz=='3',pd.cut(dff['hou_cost'],bins=unit_afford['3'],labels=inc_lbls),\
                  np.where(dff.HHSz=='4',pd.cut(dff['hou_cost'],bins=unit_afford['4'],labels=inc_lbls),\
                  np.where(dff.HHSz=='5',pd.cut(dff['hou_cost'],bins=unit_afford['5'],labels=inc_lbls),\
                  np.where(dff.HHSz=='6',pd.cut(dff['hou_cost'],bins=unit_afford['6'],labels=inc_lbls),\
                  np.where(dff.HHSz=='7',pd.cut(dff['hou_cost'],bins=unit_afford['7'],labels=inc_lbls),\
                  np.where(dff.HHSz=='8',pd.cut(dff['hou_cost'],bins=unit_afford['8'],labels=inc_lbls),''))))))))

In [13]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,WGTP1,WGTP2,WGTP3,...,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,HHSz,hou_cost,aff_cost,unit_aff
3612,0400120,177353.6016,3.0,42.0,1.029928,1000000.0,2460.0,74.0,13.0,71.0,...,71.0,78.0,42.0,44.0,43.0,42.0,3,2460.0,4921.562444,o100_ami
3637,0400119,55307.1336,7.0,172.0,1.029928,1000000.0,1405.0,70.0,81.0,157.0,...,54.0,270.0,152.0,164.0,282.0,65.0,7,1405.0,1534.772957,50_80_ami
3639,0400116,70344.0824,2.0,177.0,1.029928,1000000.0,1808.0,244.0,245.0,293.0,...,174.0,242.0,173.0,299.0,161.0,194.0,2,1808.0,1952.048287,o100_ami


### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened

In [14]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [15]:
drop_cols = ['HINCP','NP','ADJINC','ADJHSG','GRNTP','HHSz','hou_cost']

In [16]:
table = dff.copy().drop(columns=drop_cols)

In [17]:
table = table.groupby(['GEO_ID','unit_aff']).sum().reset_index()

In [18]:
table.WGTP.sum()

246922.0

In [19]:
table.head()

,GEO_ID,unit_aff,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,WGTP6,WGTP7,...,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,aff_cost
0,0400112,30_50_ami,283.0,300.0,404.0,141.0,266.0,117.0,125.0,324.0,...,353.0,509.0,257.0,104.0,235.0,435.0,445.0,243.0,295.0,2377.897766
1,0400112,50_80_ami,1221.0,1633.0,1540.0,1061.0,968.0,1463.0,1847.0,1279.0,...,555.0,905.0,1575.0,1244.0,1422.0,643.0,1264.0,1211.0,1073.0,14047.316733
2,0400112,80_100_ami,1965.0,2372.0,2089.0,1656.0,1282.0,1419.0,1581.0,1779.0,...,2353.0,2978.0,1992.0,1189.0,1444.0,1889.0,1883.0,1727.0,2138.0,28284.693804
3,0400112,o100_ami,8250.0,7691.0,7823.0,8552.0,9099.0,8622.0,8239.0,8768.0,...,8093.0,8143.0,7822.0,7776.0,9451.0,6786.0,7589.0,8442.0,8155.0,135373.548358
4,0400112,u30_ami,118.0,113.0,34.0,122.0,117.0,35.0,201.0,117.0,...,125.0,36.0,200.0,118.0,116.0,116.0,200.0,115.0,35.0,1663.385216


In [20]:
table_2 = table.copy().groupby(['GEO_ID','unit_aff']).sum().reset_index()
table_2 = make_est(table_2)
table_2 = table_2.drop(columns=repwts)

In [21]:
table_2 = pd.pivot_table(table_2,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['unit_aff'],aggfunc=np.sum).reset_index()

In [22]:
table_2

GEO_ID        hh                                            hh_CV  \
unit_aff          30_50_ami 50_80_ami 80_100_ami o100_ami u30_ami  30_50_ami   
0         0400112     283.0    1221.0     1965.0   8250.0   118.0  32.146835   
1         0400113    1067.0    3789.0     2873.0   4416.0   282.0  15.128340   
2         0400114    2444.0    9449.0     2503.0   4001.0   516.0  10.092866   
3         0400115    1972.0    9506.0     4765.0   2899.0   170.0  10.029629   
4         0400116    4302.0   13619.0     3836.0   4054.0  2085.0   7.679762   
5         0400117    3110.0    9709.0     4790.0   5627.0   651.0  10.267407   
6         0400118    5836.0   12137.0     5550.0   6951.0  2973.0   6.372659   
7         0400119    2125.0    3980.0     2533.0    993.0  1723.0   8.442734   
8         0400120     458.0    2824.0     3979.0   4902.0   342.0  26.122900   
9         0400121    2218.0    6020.0      605.0   1192.0   760.0  10.324032   
10        0400122    2710.0    8782.0     1204.0    339.0  1074.0   9.988721   
11        0400123    2777.0    8543.0     1722.0    915.0   535.0  10.299435   
12        0400125    3500.0    6714.0     2566.0    901.0  1465.0  11.221293   
13        0400128    1335.0    5958.0     2014.0   2691.0   181.0  20.837808   
14        0400129     206.0    2534.0     3030.0   2804.0    49.0  33.647076   

                                                           hh_MOE  \
unit_aff  50_80_ami 80_100_ami   o100_ami    u30_ami    30_50_ami   
0         22.910802  16.347109   6.112712  42.787027   246.182096   
1          8.372311  10.350274   7.995161  40.708814   436.804885   
2          5.714177  10.633058   8.842388  16.509424   667.494198   
3          5.641291   8.332336  10.402571  33.585426   535.209225   
4          3.745635   8.592578   6.736527  12.016254   894.025606   
5          6.116483   8.044738   7.122557  16.038323   864.078040   
6          5.295437   6.959439   6.323054   9.024398  1006.393435   
7          7.780015  10.809012  15.740568  13.839132   485.482811   
8         12.661970   8.965998   7.278279  31.874984   323.756625   
9          6.531913  24.579496  15.456577  19.838616   619.644610   
10         4.701606  22.843379  28.763326  13.626589   732.505635   
11         6.118793  12.790261  18.598412  26.668093   773.964590   
12         6.302335  11.253009  15.149772  21.504614  1062.778441   
13         8.444684  16.012451  10.785956  29.704582   752.774854   
14        10.772387   9.662984   9.368122  44.836793   187.562645   

                                                             
unit_aff    50_80_ami   80_100_ami     o100_ami     u30_ami  
0          756.985854   869.231215  1364.645094  136.623662  
1          858.423789   804.672730   955.406207  310.648570  
2         1461.070952   720.196236   957.348167  230.522555  
3         1451.136076  1074.388811   816.057417  154.501103  
4         1380.392122   891.936346   739.012199  677.964503  
5         1606.971044  1042.747711  1084.537721  282.534676  
6         1739.181705  1045.199104  1189.339835  726.013952  
7          837.906025   740.888716   422.962098  645.246925  
8          967.604291   965.393516   965.458930  294.990402  
9         1064.066282   402.402020   498.564534  407.996812  
10        1117.304235   744.249659   263.858205  396.025593  
11        1414.516432   595.997371   460.499086  386.080221  
12        1145.024167   781.370671   369.370913  852.513136  
13        1361.493842   872.668043   785.423937  145.490230  
14         738.669845   792.292780   710.824467   59.451447

In [23]:
table_2.to_excel('output/pums_rent_revised_2.xlsx')